In [1]:
import torch
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Sampler
import torch.nn.functional as F
import numpy as np
import tqdm
from torchvision import datasets, transforms
import torchvision
import pandas as pd
import os

In [3]:
from src.net import VggNetwork
from src.train import Trainer, Evaluation
from src.config import config
from PIL import Image

In [21]:
feature = 'Smiling'
size = 20000

In [50]:
data = pd.read_csv("celeba/list_attr_celeba.csv")[['image_id', feature]]

In [51]:
data[feature] = ((data[feature] + 1) / 2).astype(int)

In [52]:
negative = data[data[feature] == 0]
positive = data[data[feature] == 1]

In [53]:
size = min(size, len(negative))

In [54]:
negative = negative.sample(size)

In [55]:
size = min(size, len(positive))

In [56]:
positive = positive.sample(size)

In [62]:
split = int(len(positive) * 0.8)
train_positive, val_positive = positive.iloc[:split], positive.iloc[split:]
train_negative, val_negative = negative.iloc[:split], negative.iloc[split:]
train = pd.concat([train_positive, train_negative]).rename(columns={feature: "target"})
val = pd.concat([val_positive, val_negative]).rename(columns={feature: "target"})

In [63]:
train.to_csv("data/train.csv", index=False)
val.to_csv("data/val.csv", index=False)

In [70]:
class CelebaData(torch.utils.data.Dataset):
    def __init__(self, csv_file, folder="celeba/", image_folder="img"):
        self.img_path = os.path.join(folder, image_folder)
        self.data = pd.read_csv(csv_file)
        self.transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        imid, target = self.data.iloc[idx]
        image = Image.open(os.path.join(self.img_path, imid))
        image = self.transforms(image)
        return image, torch.LongTensor(np.array([target]))

In [79]:
loss = torch.nn.CrossEntropyLoss()

In [126]:
loss(torch.FloatTensor(1, 2),  torch.LongTensor([1]))

tensor(0.6931)

tensor([[1]])